In [1]:
import pandas as pd
from dotenv import dotenv_values, load_dotenv
import requests
import json

In [2]:
pd.__version__

'2.1.1'

## German Census Database (Genesis) – Create Table of Contents via API
  
*creating the complete Excel file may take up to 90 minutes*

for full api documentation (in German), see  
https://ergebnisse2011.zensus2022.de/datenbank/misc/ZENSUS-Webservices_Einfuehrung.pdf

code inspiration thanks to  
https://github.com/sjockers/genesis-api-example

In [3]:
# set preferred language (de|en)
langPref = "en"
# expand text output
pd.set_option('display.max_colwidth', None)

In [4]:
# Load credentials from .env file
load_dotenv()  
USERNAME, PASSWORD = dotenv_values().values()

# Set base path for API calls
BASE_URL = 'https://ergebnisse2011.zensus2022.de/api/rest/2020/'

### Check login credentials

In [5]:
hello = requests.get(BASE_URL + 'helloworld/logincheck', params={
    'username': USERNAME,
    'password': PASSWORD,
    'language': langPref
})
hello.json()["Status"]

'You have been logged in and out successfully!'

### Catalogue Service

In [6]:
variables = requests.get(BASE_URL + 'catalogue/variables', params={
    'username': USERNAME,
    'password': PASSWORD,
    'language': langPref,
    'pagelength': 200
})
variableCodes = variables.json()["List"]

In [7]:
len(variableCodes)

134

In [8]:
variableCodes[-3:]

[{'Code': 'WHGFL3',
  'Content': 'Floor area of the dwelling (20m² intervals)',
  'Type': 'Subject',
  'Values': '10',
  'Information': 'true'},
 {'Code': 'WHGNZ1',
  'Content': 'Type of use of the dwelling',
  'Type': 'Subject',
  'Values': '5',
  'Information': 'true'},
 {'Code': 'WHGRM2',
  'Content': 'Rooms',
  'Type': 'Subject',
  'Values': '7',
  'Information': 'true'}]

In [9]:
def metaVal(var):

    metadata = requests.get(BASE_URL + 'catalogue/values2variable', params={
        'username': USERNAME,
        'password': PASSWORD,
        'language': langPref,
        'pagelength': 15000,  # there are 11.340 municipality names as of 2011
        'name': var
    })
    
    return(metadata.json()["List"])

In [10]:
metadataValues = pd.DataFrame()

In [11]:
for obj in variableCodes:

    myList = metaVal(obj["Code"])

    try:
        len(myList)
    except:
        myList = [{"Code":"","Content":""}]
    
    for item in myList:
        
        metadataValues = pd.concat([metadataValues, pd.DataFrame({ "Variable": obj["Code"], \
                                                                  "VarLabel": obj["Content"], \
                                                                  "Code": item["Code"], \
                                                                  "Content": item["Content"]}, \
                                                                     index=[0])])

In [12]:
metadataValues

,Variable,VarLabel,Code,Content
0,ALTER1,Age (yearly stages),ALTERU01,Under 1 year
0,ALTER1,Age (yearly stages),ALTER001,1 year
0,ALTER1,Age (yearly stages),ALTER002,2 years
0,ALTER1,Age (yearly stages),ALTER003,3 years
0,ALTER1,Age (yearly stages),ALTER004,4 years
...,...,...,...,...
0,WHGRM2,Rooms,RAUM03,3 rooms
0,WHGRM2,Rooms,RAUM04,4 rooms
0,WHGRM2,Rooms,RAUM05,5 rooms
0,WHGRM2,Rooms,RAUM06,6 rooms


In [13]:
tables = requests.get(BASE_URL + 'catalogue/tables', params={
    'username': USERNAME,
    'password': PASSWORD,
    'language': langPref,
    'pagelength': 1500
})
tableCodes = tables.json()["List"]

In [14]:
len(tableCodes)

754

In [15]:
tableCodes[-3:]

[{'Code': '6000F-2004',
  'Content': 'Families: Size of family nucleus/Type of family nucleus\n(by family) - Equipment in dwelling/Floor area of the\ndwelling (20m² intervals)/Rooms',
  'Time': '2011-05-09 to 2011-05-09'},
 {'Code': '6000F-3001',
  'Content': 'Families: Size of private household - Senior citizen status\n- Type of private household (by family)',
  'Time': '2011-05-09 to 2011-05-09'},
 {'Code': '6000F-3002',
  'Content': 'Families: Size of family nucleus - Type of family nucleus\n(by family) - Equipment in dwelling/Floor area of the\ndwelling (20m² intervals)/Rooms',
  'Time': '2011-05-09 to 2011-05-09'}]

### Metadata Service

In [16]:
varCodes=[]

In [17]:
for item in variableCodes:
    varCodes.append(item.get("Code"))

In [18]:
varCodes[0:5]

['ALTER1', 'ALTGR1', 'ALTGR2', 'ALTGR3', 'ALTGR4']

In [19]:
def metaVar(var):

    metadata = requests.get(BASE_URL + 'metadata/variable', params={
        'username': USERNAME,
        'password': PASSWORD,
        'language': langPref,
        'name': var
    })
    
    return(metadata.json()["Object"])

In [20]:
metadataVariable = pd.DataFrame()

In [21]:
for code in varCodes:

    myObject = metaVar(code)
    
    try:
        myInformation = myObject["Information"].replace("wiki","").replace("\n"," ")
    except:
        myInformation = ""

    metadataVariable = pd.concat([metadataVariable, pd.DataFrame({"Code": myObject["Code"], \
                                                          "Content": myObject["Content"], \
                                                          "Values" : myObject["Values"], \
                                                          "Information": myInformation, \
                                                          "Updated": myObject["Updated"]},\
                                                          index=[0])])

In [22]:
metadataVariable.head(3)

,Code,Content,Values,Information,Updated
0,ALTER1,Age (yearly stages),101,The age of the person refers to the completed year of life on 9 May 2011. Broken down into individual years of age.,2022-05-09 15:37:41h
0,ALTGR1,Age (five years age groups),19,The age of the person refers to the completed year of life on 9 May 2011. This variable indicates a person's age group for reference date 9 May 2011. The ages are cited in five years age groups.,2020-10-28 10:42:15h
0,ALTGR2,Age (ten years age groups),9,The age of the person refers to the completed year of life on 9 May 2011. This variable indicates a person's age group for reference date 9 May 2011. The ages are cited in ten years age groups.,2020-10-28 10:42:19h


In [23]:
tabCodes=[]

In [24]:
for item in tableCodes:
    tabCodes.append(item.get("Code"))

In [25]:
tabCodes[-5:]

['6000F-2002', '6000F-2003', '6000F-2004', '6000F-3001', '6000F-3002']

In [26]:
def metaTab(tab):

    metadata = requests.get(BASE_URL + 'metadata/table', params={
        'username': USERNAME,
        'password': PASSWORD,
        'language': langPref,
        'name': tab
    })
    
    return(metadata.json()["Object"])

In [27]:
metadataTable = pd.DataFrame()

In [28]:
# get metadata for all tables (754 for the 2011 census) - this might take more than an hour

for code in tabCodes:

    myObject = metaTab(code)
    
    try:
        myCol2= myObject["Structure"]["Columns"][1]["Code"]
    except:
        myCol2= ""
    try:
        myCol3= myObject["Structure"]["Columns"][2]["Code"]
    except:
        myCol3= ""
    
    try:
        myRow2= myObject["Structure"]["Rows"][0]["Structure"][0]["Code"]
    except:
        myRow2= ""     
    try:
        myRow3= myObject["Structure"]["Rows"][0]["Structure"][0]["Structure"][0]["Code"]
    except:
        myRow3= ""
    try:
        myRow4= myObject["Structure"]["Rows"][0]["Structure"][0]["Structure"][0]["Structure"][0]["Code"]
    except:
        myRow4= ""
    try:
        myRow5= myObject["Structure"]["Rows"][0]["Structure"][0]["Structure"][0]["Structure"][0]["Structure"][0]["Code"]
    except:
        myRow5= ""
        
    metadataTable = pd.concat([metadataTable, pd.DataFrame({"Code": myObject["Code"], \
                                                          "Content": myObject["Content"], \
                                                          "Col1": myObject["Structure"]["Columns"][0]["Code"], \
                                                          "Col2": myCol2, \
                                                          "Col3": myCol3, \
                                                          "Row1": myObject["Structure"]["Rows"][0]["Code"], \
                                                          "Row2": myRow2, \
                                                          "Row3": myRow3, \
                                                          "Row4": myRow4, \
                                                          "Row5": myRow5, \
                                                          "Updated": myObject["Updated"]},\
                                                          index=[0])])

In [29]:
metadataTable.head(3)

,Code,Content,Col1,Col2,Col3,Row1,Row2,Row3,Row4,Row5,Updated
0,1000A-0001,Persons: Official population and area (municipalities),PRS018,FLC001,PRS017,GEOGM1,,,,,2022-01-20 08:17:38h
0,1000A-1001,Persons: Age (ten years age groups),GEODL1,GEOBL1,,ALTGR2,,,,,2022-03-21 13:56:14h
0,1000A-1002,Persons: Age (eleven classes of years),GEODL1,GEOBL1,,ALTKL2,,,,,2022-03-21 13:56:14h


## Export to a single styled Excel file

In [30]:
from openpyxl.styles import Alignment
from openpyxl.styles import DEFAULT_FONT
from openpyxl.styles import Font

In [31]:
_font = Font(name="Arial", sz=8)
defs = {k: setattr(DEFAULT_FONT, k, v) for k, v in _font.__dict__.items()}

In [33]:
with pd.ExcelWriter("census2011_toc_"+langPref+".xlsx") as writer:  

    metadataVariable.to_excel(writer, sheet_name='variables', index=False)
    metadataValues.to_excel(writer, sheet_name='values', index=False)
    metadataTable.to_excel(writer, sheet_name='tables', index=False)

    # some styling, could be skipped
    
    writer.book["variables"].column_dimensions["A"].width = 15
    writer.book["variables"].column_dimensions["B"].width = 25
    writer.book["variables"].column_dimensions["D"].width = 60
    writer.book["variables"].column_dimensions["E"].width = 20

    for idx in enumerate(metadataVariable["Code"]):
        
        writer.book["variables"].row_dimensions[idx[0]+2].height = 50
        writer.book["variables"]["B"+str(idx[0]+2)].alignment = Alignment(wrap_text=True, vertical="bottom")
        writer.book["variables"]["D"+str(idx[0]+2)].alignment = Alignment(wrap_text=True, vertical="bottom")

    writer.book["values"].column_dimensions["A"].width = 15
    writer.book["values"].column_dimensions["B"].width = 50
    writer.book["values"].column_dimensions["C"].width = 25
    writer.book["values"].column_dimensions["D"].width = 50

    writer.book["tables"].column_dimensions["A"].width = 15
    writer.book["tables"].column_dimensions["B"].width = 50
    writer.book["tables"].column_dimensions["K"].width = 20

    for idx in enumerate(metadataTable["Code"]):
        
        writer.book["tables"].row_dimensions[idx[0]+2].height = 35
        writer.book["tables"]["B"+str(idx[0]+2)].alignment = Alignment(wrap_text=True, vertical="bottom")